<a href="https://colab.research.google.com/github/MarKr17/BloodVessels/blob/master/Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pomocnicze

Funkcje pomocnicze obliczające: współczynnik Dice'a/Sorensena(dice_coef), intersection over union(iou), służą one w późniejszych etapach do obliczania współczynnikia Jaccarda. Są one potrzebne m.in do określania poprawności przewidzianych granic

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import backend as K

def iou(y_true, y_pred):
    def f(y_true, y_pred):
        intersection = (y_true * y_pred).sum()
        union = y_true.sum() + y_pred.sum() - intersection
        x = (intersection + 1e-15) / (union + 1e-15)
        x = x.astype(np.float32)
        return x
    return tf.numpy_function(f, [y_true, y_pred], tf.float32)

smooth = 1e-15
def dice_coef(y_true, y_pred):
    y_true = tf.keras.layers.Flatten()(y_true)
    y_pred = tf.keras.layers.Flatten()(y_pred)
    intersection = tf.reduce_sum(y_true * y_pred)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)

def dice_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)

# Tworzenie modelu

In [ ]:
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Input, Conv2DTranspose, Concatenate
from tensorflow.keras.models import Model



Deklaracja funkcji odpowiadających za tworzenie warstw sieci unet. Dokonano dosyć znacznego zmniejszenia pierwotnych warstw z powodu ograniczeń narzędzia GoggleColaboratory

In [ ]:
def conv_block(inputs, num_filters):
  x = Conv2D(num_filters, 3, padding="same")(inputs)
  x = BatchNormalization()(x)
  x = Activation("relu")(x)

  x = Conv2D(num_filters, 3, padding="same")(x)
  x = BatchNormalization()(x)
  x = Activation("relu")(x)
  return x
def encoder_block(inputs, num_filters):
  x = conv_block(inputs, num_filters)
  p = MaxPool2D((2,2))(x)
  return x,p
def decoder_block(inputs, skip_features, num_filters):
  x = Conv2DTranspose(num_filters, (2,2), strides=2, padding="same")(inputs)
  x = Concatenate()([x, skip_features])
  x=conv_block(x,num_filters)
  return x
def build_unet(input_shape):
  inputs = Input(input_shape)

  s1, p1 = encoder_block(inputs, 64)
  s2, p2 = encoder_block(p1, 128)
  s3, p3 = encoder_block(p2, 256)
  s4, p4 = encoder_block(p3, 512)

  b1 = conv_block(p4, 1024)

  d1 = decoder_block(b1, s4, 512)
  d2 = decoder_block(d1, s3, 256)
  d3 = decoder_block(d2, s2, 128)
  d4 = decoder_block(d3, s1, 64)

  outputs = Conv2D(1, 1, padding="same", activation="sigmoid")(d4)

  model = Model(inputs, outputs, name="UNET")
  return model
  '''
  s1, p1 = encoder_block(inputs, 32)
  s2, p2 = encoder_block(p1, 64)
  s3, p3 = encoder_block(p2, 128)

  b1 = conv_block(p3, 256)
  d1 = decoder_block(b1, s3, 128)
  d2 = decoder_block(d1, s2, 64)
  d3 = decoder_block(d2, s1, 32)
  '''

Inicjacja modelu na podstawie zbudowanej sieci Unet

In [ ]:
if __name__=="__main__":
  input_shape = (512, 512, 3)
  model = build_unet(input_shape)
  model.summary()

Model: "UNET"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_40 (InputLayer)          [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_685 (Conv2D)            (None, 512, 512, 64  1792        ['input_40[0][0]']               
                                )                                                                 
                                                                                                  
 batch_normalization_648 (Batch  (None, 512, 512, 64  256        ['conv2d_685[0][0]']             
 Normalization)                 )                                                              

# Przygotowanie danych do uczenia

Deklaracja funckji odpowiadających za wczytywanie obrazów do uczenia modelu.

In [ ]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import numpy as np
import cv2
from glob import glob
from sklearn.utils import shuffle
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau, EarlyStopping, TensorBoard
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Recall, Precision

H = 512
W = 512

def create_dir(path):
  if not os.path.exists(path):
    os.makedirs(path)
def load_data(path):
  x=sorted(glob(os.path.join(path, "image", "*.jpg")))
  y=sorted(glob(os.path.join(path, "1st", "*.png")))
  return x,y
def shuffling(x,y):
  x,y =shuffle(x,y, random_state=42)
  return x,y
def read_image(path):
  path = path.decode()
  x = cv2.imread(path,cv2.IMREAD_COLOR)
  x = x/255.0
  x = x.astype(np.float32)
  return x
def read_mask(path):
  path = path.decode()
  x = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
  x = x/255.0
  x = x.astype(np.float32)
  x = np.expand_dims(x, axis=-1)
  return x

def tf_parse(x, y):
    def _parse(x, y):
        x = read_image(x)
        y = read_mask(y)
        return x, y

    x, y = tf.numpy_function(_parse, [x, y], [tf.float32, tf.float32])
    x.set_shape([H, W, 3])
    y.set_shape([H, W, 1])
    return x, y

def tf_dataset(X, Y, batch_size=2):
    dataset = tf.data.Dataset.from_tensor_slices((X, Y))
    dataset = dataset.map(tf_parse)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(4)
    return dataset



In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

create_dir("/content/drive/MyDrive/Informatyka w medycynie/files")


Tworzenie folderów do zapisywania danych generowanych podczas uczenia modelu, deklaracja głównych parametrów(batch size, lr, num_epochs)

In [ ]:
batch_size =2
lr = 1e-4
num_epochs = 40
model_path = os.path.join("/content/drive/MyDrive/Informatyka w medycynie/files", "model.h5")
csv_path = os.path.join("/content/drive/MyDrive/Informatyka w medycynie/files", "data.csv")

In [ ]:
dataset_path = "/content/drive/MyDrive/Informatyka w medycynie/new_data"
train_path = os.path.join(dataset_path,"training")
valid_path = os.path.join(dataset_path,"test")


Wczytywanie danych

In [ ]:
train_x, train_y = load_data(train_path)
train_x, train_y = shuffling(train_x, train_y)
valid_x, valid_y = load_data(valid_path)

print(f"Train {len(train_x)}-{len(train_y)}")
print(f"Valid {len(valid_x)}-{len(valid_y)}")


Train 70-70
Valid 14-14


In [ ]:
train_dataset = tf_dataset(train_x, train_y, batch_size=batch_size)
valid_dataset = tf_dataset(valid_x, valid_y, batch_size=batch_size)


Obliczanie ile króków będzie w każdej epok(epoch)

In [ ]:
train_steps = len(train_x)//batch_size
valid_steps = len(valid_x)//batch_size

if len(train_x) % batch_size !=0:
  train_steps+=1

if len(valid_x) % batch_size !=0:
  valid_steps+=1

# Model

Tworzenie modelu

In [ ]:
model = build_unet((H, W, 3))
model.compile(loss=dice_loss, optimizer=Adam(lr), metrics=[dice_coef, iou, Recall(), Precision()])
model.summary()

Model: "UNET"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_41 (InputLayer)          [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_704 (Conv2D)            (None, 512, 512, 64  1792        ['input_41[0][0]']               
                                )                                                                 
                                                                                                  
 batch_normalization_666 (Batch  (None, 512, 512, 64  256        ['conv2d_704[0][0]']             
 Normalization)                 )                                                              

Deklaracja czynności do wykonania po kązdej epoce: obliczanie funckji straty(val_loss), zapisywanie danych do pliku csv, aktywowanie ,,early stopping'' który odpowiada za przerwanie jeżeli podczas uczenia nie widać już poprawy

In [ ]:
callbacks = [
        ModelCheckpoint(model_path, verbose=1, save_best_only=True),
        ReduceLROnPlateau(monitor="val_loss", factor=0.1, patience=5, min_lr=1e-6, verbose=1),
        CSVLogger(csv_path),
        TensorBoard(),
        EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=False)
    ]

In [ ]:
model.fit(
        train_dataset,
        epochs=num_epochs,
        validation_data=valid_dataset,
        steps_per_epoch=train_steps,
        validation_steps=valid_steps,
        callbacks=callbacks
    )

Epoch 1/40
35/35 [==============================] - ETA: 0s - loss: 0.7780 - dice_coef: 0.2220 - iou: 0.1261 - recall_13: 0.8254 - precision_11: 0.1673
Epoch 1: val_loss improved from inf to 0.86648, saving model to /content/drive/MyDrive/Informatyka w medycynie/files/model.h5
35/35 [==============================] - 32s 830ms/step - loss: 0.7780 - dice_coef: 0.2220 - iou: 0.1261 - recall_13: 0.8254 - precision_11: 0.1673 - val_loss: 0.8665 - val_dice_coef: 0.1335 - val_iou: 0.0715 - val_recall_13: 0.6298 - val_precision_11: 0.0792 - lr: 1.0000e-04
Epoch 2/40
35/35 [==============================] - ETA: 0s - loss: 0.6155 - dice_coef: 0.3845 - iou: 0.2389 - recall_13: 0.7521 - precision_11: 0.4773
Epoch 2: val_loss did not improve from 0.86648
35/35 [==============================] - 21s 585ms/step - loss: 0.6155 - dice_coef: 0.3845 - iou: 0.2389 - recall_13: 0.7521 - precision_11: 0.4773 - val_loss: 0.8695 - val_dice_coef: 0.1305 - val_iou: 0.0698 - val_recall_13: 0.0000e+00 - val_pre

# Efekty

In [ ]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import numpy as np
import pandas as pd
import cv2
from glob import glob
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.utils import CustomObjectScope
from sklearn.metrics import accuracy_score, f1_score, jaccard_score, precision_score, recall_score

Deklaracja funkcji odpowiadających za zapisywanie i wczytywanie plików

In [ ]:
H = 512
W = 512

def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

def read_image(path):
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    # x = cv2.resize(x, (W, H))
    ori_x = x
    x = x/255.0
    x = x.astype(np.float32)
    return ori_x, x

def read_mask(path):
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)  ## (512, 512)
    # x = cv2.resize(x, (W, H))
    ori_x = x
    x = x/255.0
    x = x.astype(np.int32)
    return ori_x, x

def load_data(path):
    x = sorted(glob(os.path.join(path, "image", "*.jpg")))
    y = sorted(glob(os.path.join(path, "1st", "*.png")))
    return x, y

def save_results(ori_x, ori_y, y_pred, save_image_path):
    line = np.ones((H, 10, 3)) * 255

    ori_y = np.expand_dims(ori_y, axis=-1)
    ori_y = np.concatenate([ori_y, ori_y, ori_y], axis=-1)

    y_pred = np.expand_dims(y_pred, axis=-1)
    y_pred = np.concatenate([y_pred, y_pred, y_pred], axis=-1) * 255

    cat_images = np.concatenate([ori_x, line, ori_y, line, y_pred], axis=1)
    cv2.imwrite(save_image_path, cat_images)


Kod odpowiadający za zapisywanie wyników i testowanie modelu.

In [ ]:
if __name__ == "__main__":
    """ Save the results in this folder """
    create_dir("/content/drive/MyDrive/Informatyka w medycynie/results")

    """ Load the model """
    with CustomObjectScope({'iou': iou, 'dice_coef': dice_coef, 'dice_loss': dice_loss}):
        model = tf.keras.models.load_model("/content/drive/MyDrive/Informatyka w medycynie/files/model.h5")

    """ Load the dataset """
    dataset_path = os.path.join("/content/drive/MyDrive/Informatyka w medycynie/new_data", "test")
    test_x, test_y = load_data(dataset_path)

    """ Make the prediction and calculate the metrics values """
    SCORE = []
    for x, y in tqdm(zip(test_x, test_y), total=len(test_x)):
        """ Extracting name """
        name = x.split("/")[-1].split(".")[0]

        """ Read the image and mask """
        ori_x, x = read_image(x)
        ori_y, y = read_mask(y)

        """ Prediction """
        y_pred = model.predict(np.expand_dims(x, axis=0))[0]
        y_pred = y_pred > 0.5
        y_pred = y_pred.astype(np.int32)
        y_pred = np.squeeze(y_pred, axis=-1)

        """ Saving the images """

        save_image_path = f"/content/drive/MyDrive/Informatyka w medycynie/results/{name}.png"
        save_results(ori_x, ori_y, y_pred, save_image_path)

        """ Flatten the array """
        y = y.flatten()
        y_pred = y_pred.flatten()

        """ Calculate the metrics """
        acc_value = accuracy_score(y, y_pred)
        f1_value = f1_score(y, y_pred, labels=[0, 1], average="binary")
        jac_value = jaccard_score(y, y_pred, labels=[0, 1], average="binary")
        recall_value = recall_score(y, y_pred, labels=[0, 1], average="binary")
        precision_value = precision_score(y, y_pred, labels=[0, 1], average="binary")
        SCORE.append([name, acc_value, f1_value, jac_value, recall_value, precision_value])

    score = [s[1:] for s in SCORE]
    score = np.mean(score, axis=0)
    print(f"Accuracy: {score[0]:0.5f}")
    print(f"F1: {score[1]:0.5f}")
    print(f"Jaccard: {score[2]:0.5f}")
    print(f"Recall: {score[3]:0.5f}")
    print(f"Precision: {score[4]:0.5f}")

    """ Saving """
    df = pd.DataFrame(SCORE, columns=["Image", "Acc", "F1", "Jaccard", "Recall", "Precision"])
    df.to_csv("/content/drive/MyDrive/Informatyka w medycynie/files/score.csv")

100%|██████████| 14/14 [00:06<00:00,  2.17it/s]

Accuracy: 0.96795
F1: 0.75085
Jaccard: 0.60139
Recall: 0.78026
Precision: 0.72525


# Dane

Wszystkie dane oraz notebooki użyte oraz wygenerowane w ramach projektu znajdują się w folderze:
https://drive.google.com/drive/folders/1q5aJ-41V9dM0-n6q7FVAHpp7VMf-CUN4?usp=sharing
W folderach ,,10 epok" oraz ,,20 epok" znajdują się wyniki uczenia z kolejno 10 i 20 epok.


Zauważono, że po 20 epokach z utworzonymi warstwami:

In [ ]:
s1, p1 = encoder_block(inputs, 16)
  s2, p2 = encoder_block(p1, 32)
  s3, p3 = encoder_block(p2, 64)

  b1 = conv_block(p3, 128)
  d1 = decoder_block(b1, s3, 64)
  d2 = decoder_block(d1, s2, 32)
  d3 = decoder_block(d2, s1, 16)

Wyniki są bardzo słabe, dlatego przy następnym uczeniu, wykorzystano warstwy i przeporwadzono je przez 10 epok:

In [ ]:
s1, p1 = encoder_block(inputs, 32)
  s2, p2 = encoder_block(p1, 64)
  s3, p3 = encoder_block(p2, 128)

  b1 = conv_block(p3, 256)
  d1 = decoder_block(b1, s3, 128)
  d2 = decoder_block(d1, s2, 64)
  d3 = decoder_block(d2, s1, 32)
Early stopping patience zmniejszono do 5

5 warstw:

In [ ]:
s1, p1 = encoder_block(inputs, 64)
  s2, p2 = encoder_block(p1, 128)
  s3, p3 = encoder_block(p2, 256)
  s4, p4 = encoder_block(p3, 512)
  s5, p5 = encoder_block(p4, 1024)

  b1 = conv_block(p5, 2048)

  d1 = decoder_block(b1, s5, 1024)
  d2 = decoder_block(d1, s4, 512)
  d3 = decoder_block(d2, s3, 256)
  d4 = decoder_block(d3, s2, 128)
  d5 = decoder_block(d4, s1, 64)
  outputs = Conv2D(1, 1, padding="same", activation="sigmoid")(d5)


Najlepsze uzyskane wyniki dla 4 warstw i 20 epok, drugie najlepsze dla 5 warstw i 20 epok, trzecie- 4 warstw 20 epok.